In [1]:
    import sys
!{sys.executable} -m pip install timm
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install imagesize


[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [15]:
import random
import torch

# everything should be deterministic
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
import timm
model = timm.create_model('densenet121', pretrained=True, num_classes=2)
model = model.to(device)
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNormAct2d(
      64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): DenseBlock(
      (denselayer1): DenseLayer(
        (norm1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
  

In [18]:
import pandas as pd

train_path = "work/utils/train.csv"
valid_path = "work/utils/valid.csv"

train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)


In [5]:
from utils import *

mean, std = caclulate_mean_std(train)
print(mean, std)

[0.20610136 0.20610136 0.20610136] [0.17719613 0.17719613 0.17719613]


In [19]:
from dataloading import *

eval_set = CustomVisionDataset(valid, mean, std, mode='test')

In [20]:
from torch.utils.data import DataLoader

eval_loader = DataLoader(eval_set, batch_size=100, shuffle=True, drop_last=True)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

y_true = []
y_pred = []
paths = []

with torch.no_grad():
    for sample in eval_loader:
        y_true.extend(sample[1])
        tensor = sample[0]
        tensor = tensor.to(device)
        out = model(tensor)
        probs = torch.nn.functional.softmax(out, dim=1)
        pred_labels = torch.argmax(probs, dim=1)
        y_pred.extend(pred_labels.tolist())
        paths.extend(sample[2])
        
        

        
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
        
y_true =list(map(lambda x: x.tolist(), y_true))

dict = {'path': paths, 'prediction': y_pred, 'ground_truth': y_true}
df = pd.DataFrame(dict)
df.to_csv('results.csv')        

Accuracy: 0.5245205479452055
Precision: 0.5417597554519763
Recall: 0.5245205479452055
F1 Score: 0.5299612816271133
